<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Conectar-área-de-trabajo" data-toc-modified-id="Conectar-área-de-trabajo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Conectar área de trabajo</a></span></li><li><span><a href="#Trabajar-con-datastores" data-toc-modified-id="Trabajar-con-datastores-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trabajar con datastores</a></span><ul class="toc-item"><li><span><a href="#Ver-datastores" data-toc-modified-id="Ver-datastores-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ver datastores</a></span></li><li><span><a href="#Actualizamos-los-datos-de-nuestro-data-store" data-toc-modified-id="Actualizamos-los-datos-de-nuestro-data-store-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Actualizamos los datos de nuestro data store</a></span></li></ul></li><li><span><a href="#Trabajar-con-datasets" data-toc-modified-id="Trabajar-con-datasets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Trabajar con datasets</a></span><ul class="toc-item"><li><span><a href="#Crear-un-dataset-tabular" data-toc-modified-id="Crear-un-dataset-tabular-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Crear un dataset tabular</a></span></li><li><span><a href="#Crear-un-file-Dataset" data-toc-modified-id="Crear-un-file-Dataset-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Crear un file Dataset</a></span></li><li><span><a href="#Registrar-datasets" data-toc-modified-id="Registrar-datasets-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Registrar datasets</a></span></li><li><span><a href="#Entrenar-un-modelo-con-un-dataset-tabular" data-toc-modified-id="Entrenar-un-modelo-con-un-dataset-tabular-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Entrenar un modelo con un dataset tabular</a></span></li><li><span><a href="#Registrar-un-modelo-entrenado" data-toc-modified-id="Registrar-un-modelo-entrenado-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Registrar un modelo entrenado</a></span></li></ul></li></ul></div>

# Datos

Azure ML permite gestionar los datos centralizandolos en la nube y haciendolos accesibles por los equipos de Data Science que están ejecutando experimentos y entrenando modelos.

En este notebook trabajaremos con *datastores* y *datasets*

## Conectar área de trabajo

In [1]:
import azureml.core
from azureml.core import Workspace

# Cargar el área de trabajo del fichero de configuración
ws = Workspace.from_config()
print('Versión de Azure ML {} y área de trabajo {}'.format(azureml.core.VERSION, ws.name))

Versión de Azure ML 1.32.0 y área de trabajo aml-nuclio


## Trabajar con datastores

En Azure ML, **datastores** son referencias de localizaciones de almacenamiento como un Azure Storage blob containers. Todos las áreas de trabajo tienen por defecto un datastore. Si se necesita trabajar con datos que estén guardados en diferentes localizaciones, se pueden añadir nuevos datastores a tu área de trabajo y ponerlos por defecto.

### Ver datastores

In [2]:
# Conseguir los datastores que hay por defecto
default_ds = ws.get_default_datastore()

# Enumera los datastores e indica cual es el de por defecto
for ds_name in ws.datastores:
    print(ds_name, "- Por defecto =", ds_name == default_ds.name)

azureml_globaldatasets - Por defecto = False
workspacefilestore - Por defecto = False
workspaceblobstore - Por defecto = True


In [6]:
default_ds.upload_files()

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-8b4996d3-aba4-4b8e-bec2-f0a5e9f488c7",
  "account_name": "amlnuclio7384818405",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

También se pueden ver y gestionar los datastores desde el área de trabajo de [Azure Machine Learning studio](https://ml.azure.com).

### Actualizamos los datos de nuestro data store

Se pueden subir ficheros locales al datastores para que sean accesibles para los experimentos que se vayan a ejecutar en el área de trabajo.

In [4]:
default_ds.upload_files(files=['./data/winequality.csv'], # Actualizamos el csv in /data
                       target_path='wine-data/', # Lo ponemos en una ruta del datastore
                       
                       overwrite=True, # Sobreescribimos el fcihero existente con el mismo nombre
                       show_progress=True)

Uploading an estimated of 1 files
Uploading ./data/winequality.csv
Uploaded ./data/winequality.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_abed167fe8214887881f142958c32bcc

## Trabajar con datasets

Azure Machine Learning nos provee de abstracciones de los datos en forma de *datasets*. Un dataset es una referencia versionada que especifica un conjunto de datos que se quiere usar en un experimento. El dataset puede ser *tabular* o *file*-based.

### Crear un dataset tabular

Se va a crear un dataset tabular con los datos del vino que hemos cargado en el datastore.

In [7]:
from azureml.core import Dataset

# Conseguir el datastore que hay por defecto
default_ds = ws.get_default_datastore()

#Crear un dataset tabular con la ruta del datastore
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'wine-data/*.csv'), separator=';')

# Visualizar las primeras 20 filas en un dataframe de pandas
tab_data_set.take(20).to_pandas_dataframe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,top_quality
0,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
5,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,0
7,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
8,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,0


Como se ve, es fácil convertir un dataset tabular en un dataframe de pandas.

### Crear un file Dataset

El dataset que se ha creado es un dataset *tabular* que puede ser leído como un dataframe. Esto funciona bien cuando los datos son tabulados, pero en algunos escenarios de machine learning, quizás se necesita trabajar con datos no estructurados. Por estos casos, podemos usar *file* dataset, que consiste en crear una lista de ficheros en mount virtual, los cuales puedes leer los datos.

In [8]:
#Crear un file dataset en una ruta del datastore
file_data_set = Dataset.File.from_files(path=(default_ds, 'wine-data/*.csv'))

# Conseguir los ficheros del dataset
for file_path in file_data_set.to_path():
    print(file_path)

/winequality.csv


### Registrar datasets

Ahora que se ha creado datasets referenciados a los datos de vino, se puede registrar este dataset para hacerlo más fácilmente consumible para los experimentos en el área de trabajo.

In [9]:
# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='wine-dataset',
                                        description='datos del vino',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Register the file dataset
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='wine-file-dataset',
                                            description='ficheros del vino',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registrados')

Datasets registrados


También se puede visualizar y gestionar en la pestaña de **Conjunto de datos** de tu área de trabajo de [Azure Machine Learning studio](https://ml.azure.com).

In [10]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 wine-file-dataset version 1
	 wine-dataset version 1
	 vino version 1


La posibilidad de versionar los datasets, permite cambiar los datasets sin romper los experimentos o pipelines existentes. Por defecto, se selecciona la última versión del dataset, pero se puede especificar la versiona que nos interesa:

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### Entrenar un modelo con un dataset tabular

Podemos pasar datasets a los scripts como *inputs*.

In [11]:
import os

# Crear una carpeta para un experimento
experiment_folder = 'vino_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'Carpeta creada')

vino_training_from_tab_dataset Carpeta creada


In [20]:
%%writefile $experiment_folder/vino_training.py
# Importar liberías
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Conseguir los argumentos del script (ratio de regularización e ID del dataset de entrenamiento)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Parametro de regularizacion
reg = args.reg_rate

# Contexto de la ejecución del experimento
run = Run.get_context()

# Cargar los datos
print("Cargando los datos...")
data = run.input_datasets['training_data'].to_pandas_dataframe()
# Cambiar formato de la variable
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')
# Eliminar nulos
data = data.dropna()

# Separar features and target
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
X, y = data[features].values, data['top_quality'].values

# Separar train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Entrenamiento de una regresión logística
print('Entrenamiento de una Regresión Logística con ratio de regularización de', reg)
run.log('Ratio de regularizacion',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calcular el accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Guardar el modelo entrenado en la carpeta outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/vino_model.pkl')

run.complete()

Overwriting vino_training_from_tab_dataset/vino_training.py


In [21]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails


# Crear el entorno de python
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Conseguir el dataset de entrenamiento
diabetes_ds = ws.datasets.get("wine-dataset")

# Crear un script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='vino_training.py',
                              arguments = ['--regularization', 0.1, # Parametro de regularizacion
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Referencia del dataset
                              environment=env) 

# ejecutar experimento
experiment_name = 'nuclio-train-vino'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'nuclio-train-vino_1629913048_29515924',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-25T17:37:30.550885Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'ac2fe417-f7d6-4d65-920b-6d8c7ee7d748'},
 'inputDatasets': [{'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'vino_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--input-data',
   'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'training_data': {'dataLocation': {'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f',
      'name': 'wine-dataset',
      'version': '1'},
     'dataPath': None,
     'uri': None},


Cuando termina el experimento en el apartado de Resultados y registros abriendo el **azureml-logs/70_driver_log.txt** puedes ver los logs y las métricas generadas.

### Registrar un modelo entrenado

Como en cualquier experimento, podemos entrenar y registrar un modelo en Azure ML

In [22]:
from azureml.core import Model

run.register_model(model_path='outputs/vino_model.pkl', model_name='vino_model',
                   tags={'Training context':'Tabular dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

vino_model version: 3
	 Training context : Tabular dataset
	 AUC : 0.7700142283878569
	 Accuracy : 0.8068493150684931


vino_model version: 2
	 Training context : Parameterized script
	 AUC : 0.7698252576116318
	 Accuracy : 0.8068493150684931


vino_model version: 1
	 Training context : Script
	 AUC : 0.7697446671335356
	 Accuracy : 0.8047945205479452


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


AutoML2c70a24fc0 version: 1




> **Más Información**: [Training with Datasets](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)